In [2]:
import cvxpy as cp
import numpy as np

N = 6; 
SINRdB = 10; # QoS constraint in dB (same for all users)
G = 3; # multicast groups
phi = np.concatenate((np.arange(2,19,4), np.arange(26,63,4)))
phi = np.concatenate((np.flip(360-phi), phi))

M = len(phi); # total users


def get_channel(random = True,
                phi = None, 
                N=6, 
                M=12, 
                L=None,
                SINRdB=10,
                G=2,
                n_beams_per_group=2,
                sigma2=1):
    
    assert M % G == 0, 'cannot divide into groups with equal number of users'
    assert G >= 1, 'Number of groups cannot be less than 1' 
    assert M % (G*n_beams_per_group) == 0, 'Number of users in a group cannot be divided equally into different beams'

    SINR = 10**(SINRdB/10)*np.ones(M)
    sigma2 = np.ones(M)*sigma2

    if phi is not None: 
        assert len(phi) == M, 'number of user locations not equal to the number of users'
    if not random: 
        if phi is not None:
            theta = -np.pi*np.sin(phi*np.pi/180) 
        else:
            phi = np.random.rand(M)*2*np.pi
            phi = np.sort(phi)
            perm = np.random.permutation(np.arange(n_beams_per_group*G))
            phi_new = np.array([])
            nbeams = n_beams_per_group*G
            for i in perm:
                phi_new = np.concatenate((phi_new, phi[int(i*M/nbeams):int((i+1)*M/nbeams)]))
            phi = phi_new.copy()
            theta = -np.pi*np.sin(phi)
    else:
        phi = np.random.rand(M)*2*np.pi
        theta = -np.pi*np.sin(phi)

    Gk = M/G*np.ones((G,1))
    Gk = Gk.astype(np.int8)
    csGk = np.cumsum(Gk).astype(np.int8)

    group_membership = np.zeros(Gk[0])
    for k in range(1,G):
        group_membership = np.concatenate((group_membership, np.ones(csGk[k] - csGk[k-1])*k))

    group_membership = group_membership.astype(np.int8)
    H = v_theta(theta, N)

    return H, group_membership


def solve_beamforming(H, G, group_membership, SINR, sigma2):
    N,M = H.shape

    V = 2*H.conj().T
    V[:,0] = 1

    r = cp.Variable((G*N), complex=True)
    X = []
    for _ in range(G):
        X.append(cp.Variable((N,N), hermitian=True))

    A = np.zeros((M, G*N)) + 1j*np.zeros((M,G*N))
    for i in range(M):
        a = -SINR[i]*sigma2[i]*np.ones(G)
        a[group_membership[i]] = 1
        A[i, :] = np.kron(a, V[i,:])

    c = np.zeros(G*N + 2*M + G*N*N)
    c[[n*N for n in range(G)]] = 1

    b = np.concatenate((SINR*sigma2, np.zeros(G*N)))
    A = np.concatenate((A, 1j*np.eye(M), -np.eye(M), np.zeros((M, G*N*N))), axis=1)

    allE = np.zeros((N, N*N))
    for n in range(N):
        E = np.diag(np.ones(N-n),-n)
        allE[n,:] = -E.flatten(order='F') # vectorize in matlab is column-major flatten
    #     allE[n,:] = -cp.vec(E)


    subA = np.concatenate((np.eye(G*N), np.zeros((G*N, 2*M)), np.kron(np.eye(G), allE)), axis=1)
    A = np.concatenate((A, subA), axis=0)

    obj = cp.Minimize(cp.real(cp.sum(cp.multiply(c[:G*N], r))))


    x = cp.vec(X[0])
    for g in range(1,len(X)):
        x = cp.hstack((x, cp.vec(X[g])))

    constraints = []

    constraints.append(cp.real(A[:M, :G*N] @ r + A[:M, G*N+2*M:] @ x) >= b[:M])
    constraints.append(A[M:, :G*N] @ r + A[M:, G*N+2*M:] @ x == 0)

    # PSD constraints
    for g in range(G):
    #     constraints.append(cp.reshape(x[g*N*N:(g+1)*N*N], (N,N)) >> 0)
        constraints.append(X[g] >> 0)

    constraints.append(cp.imag(r[[n*N for n in range(G)]]) == 0)

    prob = cp.Problem(obj, constraints)
    prob.solve(solver='MOSEK')

    # return the W matrix, z vector, objective value and optimality status
    if prob.status in ['infeasible', 'unbounded']:
        print('infeasible solution') 
        return None, None, np.inf, False
    
    rmat = np.reshape(r.value,(N,G), order='F');
    rmat = np.concatenate((np.flip(rmat[1:,:], axis=0).conj(), rmat), axis=0);

    W = np.zeros((N,G)) + 1j*np.zeros((N,G))
    for g in range(G):
        W[:,g] = spectral_factorization_fft(rmat[:,g]) 
    return W, X, prob.objective.value, True


def spectral_factorization_fft(rvec):
    epsilon = 1e-16

    L = int(2**(np.ceil(np.log2(25*len(rvec)))))
    psd = np.exp(1j*(len(rvec)-1)/2*np.linspace(0,2*np.pi*(L-1)/L,L))*np.fft.fft(rvec, n=L)

    psd[np.real(psd) < 0] = epsilon;

    if 0 >= np.min(psd):
       print('An error occurred in spectralfactorization_fft:  the PSD has negative or zero elements.')

    a=np.fft.ifft(np.log(psd))
    hh=np.fft.ifft( np.exp( np.fft.fft(np.concatenate((np.expand_dims(a[0]/2, 0),a[1:int(L/2)])),L)));
    hh = np.flip(hh[:int((len(rvec)+1)/2)]);
    return hh

# @staticmethod
def v_theta(theta, N):
    """
    Create vandermonde channels using the angles theta
    """
    powers = np.arange(N)
    powers = np.expand_dims(powers, 1)
    theta1 = np.expand_dims(theta.copy(), 0)
    H = np.exp(1j*theta1*powers)
    return H

In [10]:
import cvxpy as cp
import numpy as np
N = 6; 
SINRdB = 10; # QoS constraint in dB (same for all users)
G = 3; # multicast groups
phi = np.concatenate((np.arange(2,19,4), np.arange(26,63,4)))
phi = np.concatenate((np.flip(360-phi), phi))

M = len(phi); # total users
sigma2 = 1
n_beams_per_group = 2
random=False

H, group_membership = get_channel(random = False,
                phi = phi, 
                N=N, 
                M=M, 
                L=None,
                SINRdB=SINRdB,
                G=G,
                n_beams_per_group=n_beams_per_group,
                sigma2=sigma2)

SINR = 10**(SINRdB/10)*np.ones(M)
sigma2 = np.ones(M)*sigma2

w, X, obj, opt =  solve_beamforming(H, G, group_membership, SINR, sigma2)



In [5]:
assert M % G == 0, 'cannot divide into groups with equal number of users'
assert G >= 1, 'Number of groups cannot be less than 1' 
assert M % (G*n_beams_per_group) == 0, 'Number of users in a group cannot be divided equally into different beams'

SINR = 10**(SINRdB/10)*np.ones(M)
sigma2 = np.ones(M)*sigma2

if phi is not None: 
    assert len(phi) == M, 'number of user locations not equal to the number of users'
if not random: 
    if phi is not None:
        theta = -np.pi*np.sin(phi*np.pi/180) 
    else:
        phi = np.random.rand(M)*2*np.pi
        phi = np.sort(phi)
        perm = np.random.permutation(np.arange(n_beams_per_group*G))
        phi_new = np.array([])
        nbeams = n_beams_per_group*G
        for i in perm:
            phi_new = np.concatenate((phi_new, phi[int(i*M/nbeams):int((i+1)*M/nbeams)]))
        phi = phi_new.copy()
        theta = -np.pi*np.sin(phi)
else:
    phi = np.random.rand(M)*2*np.pi
    theta = -np.pi*np.sin(phi)

Gk = M/G*np.ones((G,1))
Gk = Gk.astype(np.int8)
csGk = np.cumsum(Gk).astype(np.int8)

group_membership = np.zeros(Gk[0])
for k in range(1,G):
    group_membership = np.concatenate((group_membership, np.ones(csGk[k] - csGk[k-1])*k))

group_membership = group_membership.astype(np.int8)
H = v_theta(theta, N)

# return H



T = 1000
L = 7
# z_sol = np.array([1,1,1,1,1,1,1,1])
z_sol = np.zeros(N)
z_mask = np.zeros(N)

z_sol[[0,1,2]] = 0
# z_mask[[0,2,4,8,10]] = 0
z_mask[[10]] = 1


# solve beamforming
import cvxpy as cp
N,M = H.shape

V = 2*H.conj().T
V[:,0] = 1

r = cp.Variable((G*N), complex=True)
z = cp.Variable(N, complex=False)
X = []
for _ in range(G):
    X.append(cp.Variable((N,N), hermitian=True))

A = np.zeros((M, G*N)) + 1j*np.zeros((M,G*N))
for i in range(M):
    a = -SINR[i]*sigma2[i]*np.ones(G)
    a[group_membership[i]] = 1
    A[i, :] = np.kron(a, V[i,:])

c = np.zeros(G*N + 2*M + G*N*N)
c[[n*N for n in range(G)]] = 1

b = np.concatenate((SINR*sigma2, np.zeros(G*N)))
A = np.concatenate((A, 1j*np.eye(M), -np.eye(M), np.zeros((M, G*N*N))), axis=1)

allE = np.zeros((N, N*N))
for n in range(N):
    E = np.diag(np.ones(N-n),-n)
    allE[n,:] = -E.flatten(order='F') # vectorize in matlab is column-major flatten
#     allE[n,:] = -cp.vec(E)


subA = np.concatenate((np.eye(G*N), np.zeros((G*N, 2*M)), np.kron(np.eye(G), allE)), axis=1)
A = np.concatenate((A, subA), axis=0)

obj = cp.Minimize(cp.real(cp.sum(cp.multiply(c[:G*N], r))))

x = cp.vec(X[0])
for g in range(1,len(X)):
    x = cp.hstack((x, cp.vec(X[g])))

constraints = []

constraints.append(cp.real(A[:M, :G*N] @ r + A[:M, G*N+2*M:] @ x) >= b[:M])
constraints.append(A[M:, :G*N] @ r + A[M:, G*N+2*M:] @ x == 0)

# PSD constraints
for g in range(G):
#     constraints.append(cp.reshape(x[g*N*N:(g+1)*N*N], (N,N)) >> 0)
    constraints.append(X[g] >> 0)

constraints.append(cp.imag(r[[n*N for n in range(G)]]) == 0)

# z related constraints
for g in range(G):
    constraints += [cp.real(cp.diag(X[g])) <= T*z]
    
    constraints += [cp.real(X[g]) <=  T*cp.reshape(z, (1, N))]
    constraints += [cp.real(X[g]) >= -T*cp.reshape(z, (1, N))]
    constraints += [cp.imag(X[g]) <=  T*cp.reshape(z, (1, N))]
    constraints += [cp.imag(X[g]) >= -T*cp.reshape(z, (1, N))]
    constraints += [cp.real(X[g]) <=  T*cp.reshape(z, (N, 1))]
    constraints += [cp.real(X[g]) >= -T*cp.reshape(z, (N, 1))]
    constraints += [cp.imag(X[g]) <=  T*cp.reshape(z, (N, 1))]
    constraints += [cp.imag(X[g]) >= -T*cp.reshape(z, (N, 1))]
    
constraints += [cp.sum(z) == L]
constraints += [z >= np.zeros(N), z <= np.ones(N)]

for n in range(N):
    if z_mask[n]:
        constraints += [z[n] == z_sol[n]]
    
prob = cp.Problem(obj, constraints)
prob.solve(solver='MOSEK')
print(prob.objective.value)

rmat = np.reshape(r.value,(N,G), order='F');
rmat = np.concatenate((np.flip(rmat[1:,:], axis=0).conj(), rmat), axis=0);

W = np.zeros((N,G)) + 1j*np.zeros((N,G))
for g in range(G):
    W[:,g] = spectral_factorization_fft(rmat[:,g]) 


10.642857269418627


In [17]:
W[:,2]

array([-0.03856194+1.28925929e-01j, -0.04372108-1.37862632e-02j,
       -0.14388011-4.14172501e-02j,  0.10135535-2.59439911e-01j,
        0.26535186+2.36601811e-01j, -0.14481966+5.57729282e-02j,
       -0.14062272+1.97763506e-01j, -0.35305567-5.49450080e-01j,
        0.95737834+1.22608085e-02j, -0.50609945+8.16153333e-01j,
       -0.39055649-6.09098602e-01j,  0.41757151+4.51326644e-07j])

In [ ]:
g

In [33]:
z.unsqueeze()

AttributeError: 'Variable' object has no attribute 'unsqueeze'

## Try without variable z

In [37]:
assert M % G == 0, 'cannot divide into groups with equal number of users'
assert G >= 1, 'Number of groups cannot be less than 1' 
assert M % (G*n_beams_per_group) == 0, 'Number of users in a group cannot be divided equally into different beams'

SINR = 10**(SINRdB/10)*np.ones(M)
sigma2 = np.ones(M)*sigma2

if phi is not None: 
    assert len(phi) == M, 'number of user locations not equal to the number of users'
if not random: 
    if phi is not None:
        theta = -np.pi*np.sin(phi*np.pi/180) 
    else:
        phi = np.random.rand(M)*2*np.pi
        phi = np.sort(phi)
        perm = np.random.permutation(np.arange(n_beams_per_group*G))
        phi_new = np.array([])
        nbeams = n_beams_per_group*G
        for i in perm:
            phi_new = np.concatenate((phi_new, phi[int(i*M/nbeams):int((i+1)*M/nbeams)]))
        phi = phi_new.copy()
        theta = -np.pi*np.sin(phi)
else:
    phi = np.random.rand(M)*2*np.pi
    theta = -np.pi*np.sin(phi)

Gk = M/G*np.ones((G,1))
Gk = Gk.astype(np.int8)
csGk = np.cumsum(Gk).astype(np.int8)

group_membership = np.zeros(Gk[0])
for k in range(1,G):
    group_membership = np.concatenate((group_membership, np.ones(csGk[k] - csGk[k-1])*k))

group_membership = group_membership.astype(np.int8)
H = v_theta(theta, N)

# return H



T = 1000
L = 7
# z_sol = np.array([1,1,1,1,1,1,1,1])

z_sol = np.zeros(N)
z_mask = np.zeros(N)

# z_sol[[0,1,2]] = 0
# z_mask[[0,2,4,8,10]] = 0
z_mask[[10]] = 1


# solve beamforming
import cvxpy as cp

# set H elements for the unselected antennas to zero
num_selected_antennas = int(N-sum(z_mask*(1-z_sol)))
H_new = np.zeros((num_selected_antennas, H.shape[1]))
n_sel = 0
for n in range(N):
    if not z_mask[n] or z_sol[n]:
        H_new[n_sel,:] = H[n,:] 
        n_sel += 1
        
H = H_new.copy()

N,M = H.shape

V = 2*H.conj().T
V[:,0] = 1

r = cp.Variable((G*N), complex=True)
# z = cp.Variable(N, complex=False)

 X = []
for _ in range(G):
    X.append(cp.Variable((N,N), hermitian=True))

A = np.zeros((M, G*N)) + 1j*np.zeros((M,G*N))
for i in range(M):
    a = -SINR[i]*sigma2[i]*np.ones(G)
    a[group_membership[i]] = 1
    A[i, :] = np.kron(a, V[i,:])

c = np.zeros(G*N + 2*M + G*N*N)
c[[n*N for n in range(G)]] = 1

b = np.concatenate((SINR*sigma2, np.zeros(G*N)))
A = np.concatenate((A, 1j*np.eye(M), -np.eye(M), np.zeros((M, G*N*N))), axis=1)

allE = np.zeros((N, N*N))
for n in range(N):
    E = np.diag(np.ones(N-n),-n)
    allE[n,:] = -E.flatten(order='F') # vectorize in matlab is column-major flatten
#     allE[n,:] = -cp.vec(E)


subA = np.concatenate((np.eye(G*N), np.zeros((G*N, 2*M)), np.kron(np.eye(G), allE)), axis=1)
A = np.concatenate((A, subA), axis=0)

obj = cp.Minimize(cp.real(cp.sum(cp.multiply(c[:G*N], r))))

x = cp.vec(X[0])
for g in range(1,len(X)):
    x = cp.hstack((x, cp.vec(X[g])))

constraints = []

constraints.append(cp.real(A[:M, :G*N] @ r + A[:M, G*N+2*M:] @ x) >= b[:M])
constraints.append(A[M:, :G*N] @ r + A[M:, G*N+2*M:] @ x == 0)

# PSD constraints
for g in range(G):
#     constraints.append(cp.reshape(x[g*N*N:(g+1)*N*N], (N,N)) >> 0)
    constraints.append(X[g] >> 0)

constraints.append(cp.imag(r[[n*N for n in range(G)]]) == 0)


prob = cp.Problem(obj, constraints)
prob.solve(solver='MOSEK')
print(prob.objective.value)

rmat = np.reshape(r.value,(N,G), order='F');
rmat = np.concatenate((np.flip(rmat[1:,:], axis=0).conj(), rmat), axis=0);

W = np.zeros((N,G)) + 1j*np.zeros((N,G))
for g in range(G):
    W[:,g] = spectral_factorization_fft(rmat[:,g]) 


/scratch/sagar/Projects/combopt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:63: ComplexWarning: Casting complex values to real discards the imaginary part


None


ValueError: cannot reshape array of size 1 into shape (11,3)

In [36]:
def solve_beamforming(H, G, group_membership, SINR, sigma2):
    N,M = H.shape
    
    X = []
    for k in range(G):
        X.append(cp.Variable((N,N), hermitian = True))
    s = cp.Variable(M, complex=False)

    obj_val = 0
    for k in range(G):
        obj_val += cp.real(cp.trace(X[k]))
    obj = cp.Minimize(obj_val)
    # obj = cp.Minimize(cp.trace(cp.real(np.sum(X, 0))))

    constraints = []

    for i in range(M):
        Qi = np.matmul(np.expand_dims(H[:,i],1), np.expand_dims(H[:,i],1).conj().T)
        k = group_membership[i]

        #     maskI = np.ones(G)
        #     maskI[k] = 0
        #     maskI = np.expand_dims(maskI, (1,2))
        mask_list = [i for i in range(G)]
        del mask_list[k]
        print(k, mask_list)
        constraints.append(cp.real(cp.trace(Qi @ X[k])) - gamma*cp.real(cp.sum([cp.trace(Qi @ X[l]) for l in mask_list])) - s[i]== gamma)
        constraints.append(s[i] >= 0)

    for k in range(G):
        constraints.append(X[k] >> 0)

    prob = cp.Problem(obj, constraints)
    prob.solve(verbose=False)
    
#     if prob.status in ['infeasible', 'unbounded']:
#         print('infeasible solution') 
#         return None, None, np.inf, False
    
#     rmat = np.reshape(r.value,(N,G), order='F');
#     rmat = np.concatenate((np.flip(rmat[1:,:], axis=0).conj(), rmat), axis=0);

#     W = np.zeros((N,G)) + 1j*np.zeros((N,G))
#     for g in range(G):
#         W[:,g] = spectral_factorization_fft(rmat[:,g]) 
#     return W, X, prob.objective.value, True

11

In [36]:
import cvxpy as cp
import numpy as np
N = 7; 
SINRdB = 10; # QoS constraint in dB (same for all users)
G = 3; # multicast groups
phi = np.concatenate((np.arange(2,19,4), np.arange(26,63,4)))
phi = np.concatenate((np.flip(360-phi), phi))

M = len(phi); # total users
sigma2 = 1
n_beams_per_group = 2
random=False

H, group_membership = get_channel(random = False,
                phi = phi, 
                N=N, 
                M=M, 
                L=None,
                SINRdB=SINRdB,
                G=G,
                n_beams_per_group=n_beams_per_group,
                sigma2=sigma2)

SINR = 10**(SINRdB/10)*np.ones(M)
sigma2 = np.ones(M)*sigma2

assert M % G == 0, 'cannot divide into groups with equal number of users'
assert G >= 1, 'Number of groups cannot be less than 1' 
assert M % (G*n_beams_per_group) == 0, 'Number of users in a group cannot be divided equally into different beams'

SINR = 10**(SINRdB/10)*np.ones(M)
sigma2 = np.ones(M)*sigma2

if phi is not None: 
    assert len(phi) == M, 'number of user locations not equal to the number of users'
if not random: 
    if phi is not None:
        theta = -np.pi*np.sin(phi*np.pi/180) 
    else:
        phi = np.random.rand(M)*2*np.pi
        phi = np.sort(phi)
        perm = np.random.permutation(np.arange(n_beams_per_group*G))
        phi_new = np.array([])
        nbeams = n_beams_per_group*G
        for i in perm:
            phi_new = np.concatenate((phi_new, phi[int(i*M/nbeams):int((i+1)*M/nbeams)]))
        phi = phi_new.copy()
        theta = -np.pi*np.sin(phi)
else:
    phi = np.random.rand(M)*2*np.pi
    theta = -np.pi*np.sin(phi)

Gk = M/G*np.ones((G,1))
Gk = Gk.astype(np.int8)
csGk = np.cumsum(Gk).astype(np.int8)

group_membership = np.zeros(Gk[0])
for k in range(1,G):
    group_membership = np.concatenate((group_membership, np.ones(csGk[k] - csGk[k-1])*k))

group_membership = group_membership.astype(np.int8)
H = v_theta(theta, N)

# return H



T = 1000
L = 5
# z_sol = np.array([1,1,1,1,1,1,1,1])

z_sol = np.zeros(N)
z_mask = np.zeros(N)

z_mask[[5]] = 1


# solve beamforming
import cvxpy as cp

N,M = H.shape

X = []
for k in range(G):
    X.append(cp.Variable((N,N), hermitian = True))
    
z = cp.Variable(N, complex=False)
s = cp.Variable(M, complex=False)

obj_val = 0
for k in range(G):
    obj_val += cp.real(cp.trace(X[k]))
obj = cp.Minimize(obj_val)
# obj = cp.Minimize(cp.trace(cp.real(np.sum(X, 0))))

constraints = []

for i in range(M):
    Qi = np.matmul(np.expand_dims(H[:,i],1), np.expand_dims(H[:,i],1).conj().T)
    k = group_membership[i]

    #     maskI = np.ones(G)
    #     maskI[k] = 0
    #     maskI = np.expand_dims(maskI, (1,2))
    mask_list = [i for i in range(G)]
    del mask_list[k]
    print(k, mask_list)
    constraints.append(cp.real(cp.trace(Qi @ X[k])) - SINR[i]*cp.real(cp.sum([cp.trace(Qi @ X[l]) for l in mask_list])) >= SINR[i])

for k in range(G):
    constraints.append(X[k] >> 0)

# z related constraints
for g in range(G):
    constraints += [cp.real(cp.diag(X[g])) <= T*z]
    
#     constraints += [cp.real(X[g]) <=  T*cp.reshape(z, (1, N))]
#     constraints += [cp.real(X[g]) >= -T*cp.reshape(z, (1, N))]
#     constraints += [cp.imag(X[g]) <=  T*cp.reshape(z, (1, N))]
#     constraints += [cp.imag(X[g]) >= -T*cp.reshape(z, (1, N))]
#     constraints += [cp.real(X[g]) <=  T*cp.reshape(z, (N, 1))]
#     constraints += [cp.real(X[g]) >= -T*cp.reshape(z, (N, 1))]
#     constraints += [cp.imag(X[g]) <=  T*cp.reshape(z, (N, 1))]
#     constraints += [cp.imag(X[g]) >= -T*cp.reshape(z, (N, 1))]
    
constraints += [cp.sum(z) == L]
constraints += [z >= np.zeros(N), z <= np.ones(N)]

for n in range(N):
    if z_mask[n]:
        constraints += [z[n] == z_sol[n]]
    
prob = cp.Problem(obj, constraints)
prob.solve(verbose=True)


0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Mar 29 12:54:51 PM: Your problem has 154 variables, 40 constraints, and 0 parameters.
(CVXPY) Mar 29 12:54:51 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 29 12:54:51 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 29 12:54:51 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation       

24.548860610366987

In [31]:
H[5,:]

array([ 2.64666993e-01+0.96433987j,  7.28451249e-01+0.68509764j,
        9.89986009e-01+0.14116551j,  8.61097185e-01-0.5084404j ,
        2.99137726e-01-0.95420995j, -4.66115733e-01-0.88472376j,
       -9.69891878e-01-0.24353592j, -8.01500330e-01+0.59799433j,
       -6.79926566e-15+1.j        ,  8.23789614e-01+0.56689565j,
        1.41165511e-01-0.98998601j, -7.90906369e-01-0.61193718j,
       -9.15545819e-01+0.40221369j, -7.10729623e-02+0.99747112j,
        8.53463971e-01+0.52115185j,  8.53463971e-01-0.52115185j,
       -7.10729623e-02-0.99747112j, -9.15545819e-01-0.40221369j,
       -7.90906369e-01+0.61193718j,  1.41165511e-01+0.98998601j,
        8.23789614e-01-0.56689565j,  1.19434012e-15-1.j        ,
       -8.01500330e-01-0.59799433j, -9.69891878e-01+0.24353592j,
       -4.66115733e-01+0.88472376j,  2.99137726e-01+0.95420995j,
        8.61097185e-01+0.5084404j ,  9.89986009e-01-0.14116551j,
        7.28451249e-01-0.68509764j,  2.64666993e-01-0.96433987j])

In [29]:
H[5,:]

array([ 2.64666993e-01+0.96433987j,  7.28451249e-01+0.68509764j,
        9.89986009e-01+0.14116551j,  8.61097185e-01-0.5084404j ,
        2.99137726e-01-0.95420995j, -4.66115733e-01-0.88472376j,
       -9.69891878e-01-0.24353592j, -8.01500330e-01+0.59799433j,
       -6.79926566e-15+1.j        ,  8.23789614e-01+0.56689565j,
        1.41165511e-01-0.98998601j, -7.90906369e-01-0.61193718j,
       -9.15545819e-01+0.40221369j, -7.10729623e-02+0.99747112j,
        8.53463971e-01+0.52115185j,  8.53463971e-01-0.52115185j,
       -7.10729623e-02-0.99747112j, -9.15545819e-01-0.40221369j,
       -7.90906369e-01+0.61193718j,  1.41165511e-01+0.98998601j,
        8.23789614e-01-0.56689565j,  1.19434012e-15-1.j        ,
       -8.01500330e-01-0.59799433j, -9.69891878e-01+0.24353592j,
       -4.66115733e-01+0.88472376j,  2.99137726e-01+0.95420995j,
        8.61097185e-01+0.5084404j ,  9.89986009e-01-0.14116551j,
        7.28451249e-01-0.68509764j,  2.64666993e-01-0.96433987j])

array([0., 0., 0., 0., 0., 0., 1.])

In [18]:
z_sol

array([0., 0., 0., 0., 0., 0., 0.])

(6, 30)

In [9]:
N

10

In [5]:
SINR

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10.])